# Plotting Temporal changes

I will plot all heights regardless of locations

## Test

One track would be plotted as an example

### Setting

In [113]:
import glob
import os
import pandas as pd
from pathlib import Path
import h5py
import re
import numpy as np
import matplotlib.pyplot as plt
import datetime
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import statistics
%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/download/')
## list them up and check them
files= list(data_home.glob('*.h5'))
## choose a file as an example

##### function of reading alt13
def alt13_to_df(filename, beam):    
    f = h5py.File(filename, 'r')
    f_beam = f[beam]
    lat = f_beam['segment_lat'][:]
    long = f_beam['segment_lon'][:]
    ws = f_beam['ht_water_surf'][:]
    ws_sd = f_beam['stdev_water_surf'][:]
    ws_err = f_beam['err_ht_water_surf'][:]
    ortho = f_beam['ht_ortho'][:]
    wd = f_beam['water_depth'][:]
    alt13_df = pd.DataFrame({'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                            'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                            'WaterD':wd})
    return alt13_df

### We set 'gt2l' as a beam of example for tracking
D_dict={}
error_count=0
for ff in files:
    try:
        D_dict[ff]=alt13_to_df(ff, 'gt2l')
    except KeyError as e:
        ##print(f'file {ff} encountered error {e}')
        error_count += 1
        
### bounds of Tonle Sap Lake
sp_ex = [103.643, 104.667, 12.375, 13.287]

### Plot of one track

##### I need to make a col and draw it repetitively

Function for record the beam, date and RGT, as well as other information.

In [3]:
### I made a function for stacking the information
def alt13_to_df_beams(filename):    
    f = h5py.File(filename, 'r')
    rgt = str(filename).split("_")[5][0:4]
    cycle = int(str(filename).split("_")[5][4:6])
    version = int(str(filename).split("_")[5][6:8])
    ymd = str(filename).split("_")[4][0:8]
    ymd_trans = datetime.datetime(int(ymd[0:4]),int(ymd[4:6]),int(ymd[6:8]))
    date = ymd_trans.strftime("%Y-%m-%d")
    beam_lst = list(f)[2:-1]
    alt13_df = pd.DataFrame()
    for beam in beam_lst:
        f_beam = f[beam]
        lat = f_beam['segment_lat'][:]
        long = f_beam['segment_lon'][:]
        ws = f_beam['ht_water_surf'][:]
        ws_sd = f_beam['stdev_water_surf'][:]
        ws_err = f_beam['err_ht_water_surf'][:]
        ortho = f_beam['ht_ortho'][:]
        wd = f_beam['water_depth'][:]
        df_beam = pd.DataFrame({'Beam': beam ,'RGT':rgt,'Cycle': cycle, 'Date':date, 'Date_num':int(ymd),'Ver.':version,
                                 'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                                'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                                'WaterD':wd})
        alt13_df = alt13_df.append(df_beam, ignore_index = True)
        
    return alt13_df


In [ ]:
test_beams = alt13_to_df_beams(files[37])

In [ ]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(test_beams['Date'],test_beams['SurfaceH'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

## Plot of all tracks with all beams

In [4]:
a_tracks = pd.DataFrame()
for ff in files:
    a_tracks = a_tracks.append(alt13_to_df_beams(ff), ignore_index = True)
print('done')

done


In [5]:
#### Sorting to draw graph
a_tracks.sort_values(by=['Date_num'], inplace=True)

In [6]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.plot(a_tracks['Date'],a_tracks['SurfaceH'],'.',markersize=0.25, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot the tracks by grouping their locations 

#### Map all tracks with labels

Drawing a just track first.

In [7]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(sp_ex, crs=ccrs.PlateCarree())
plt.scatter(a_tracks['Longitude'], a_tracks['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 10)
plt.title("Tracks on TSL")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Tracks on TSL')

Let's plot water levels according to cycles.

Before it, I will plot the third track, and see the temporal gaps

In [8]:
#### Select the 3rd track
track_3rd = a_tracks.loc[a_tracks['Cycle'] == 3]

In [9]:
#### Mapping
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(sp_ex, crs=ccrs.PlateCarree())
plt.scatter(track_3rd['Longitude'], track_3rd['Latitude'],s=1)

request = cimgt.Stamen('terrain-background')
ax.add_image(request, 10)
plt.title("Tracks on TSL")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Tracks on TSL')

In [196]:
print(track_3rd['Date_num'].max()-track_3rd['Date_num'].min())

201


It is about 201 day. Too long to see the water changes.

I will see some papers and see how they make a graph.

Here are my plans to do today.
1. Plot outliers
2. Add averaged points of the day
3. See the relationship of water level with extent
4. Newly apply to Bugon Res.

## Plot outliers and averaged heights

Outliers
IQR = q0.75 − q0.25
LSHoutlier > q0.75 + 1.5 ∗ IQR or LSHoutlier < q0.25 − 1.5 ∗ IQR

I need to do it by beams, so I need to modify the function

In [10]:
a_tracks.head()

,Beam,RGT,Cycle,Date,Date_num,Ver.,Latitude,Longitude,SurfaceH,SH_SD,SH_error,OrthoH,WaterD
0,gt1l,0461,1,2018-10-28,20181028,1,12.729695,104.512303,-8.119766,3.402823e+38,3.402823e+38,8.898558,3.402823e+38
227,gt3r,0461,1,2018-10-28,20181028,1,12.700246,104.450221,-8.321230,6.500000e-02,3.402823e+38,8.913179,3.402823e+38
226,gt3r,0461,1,2018-10-28,20181028,1,12.701124,104.450314,-8.297829,6.500000e-02,3.402823e+38,8.937248,3.402823e+38
225,gt3r,0461,1,2018-10-28,20181028,1,12.701847,104.450388,-8.328204,6.500000e-02,3.402823e+38,8.907535,3.402823e+38
224,gt3r,0461,1,2018-10-28,20181028,1,12.702500,104.450451,-8.313114,6.500000e-02,3.402823e+38,8.923317,3.402823e+38


#### Function for removing outliers

In [72]:
### outliers
def out_rmv(df,var):
    ### lower (25%)
    lwq = df[var].quantile(0.25)
    ### upper (75%)
    hwq = df[var].quantile(0.75)

    ### IQR
    iqr = hwq - lwq

    lw_out = lwq-1.5*iqr
    hw_out = hwq+1.5*iqr

    ###LSHoutlier > q0.75 + 1.5 ∗ IQR or LSHoutlier < q0.25 − 1.5 ∗ IQR
    return df.loc[(df[var] >= lw_out) & (df[var] <= hw_out)]

#### Function for stacking DB without outliers

In [99]:
### I made a function for stacking the information
def alt13_to_df_beams_out(filename):    
    f = h5py.File(filename, 'r')
    rgt = str(filename).split("_")[5][0:4]
    cycle = int(str(filename).split("_")[5][4:6])
    version = int(str(filename).split("_")[5][6:8])
    ymd = str(filename).split("_")[4][0:8]
    ymd_trans = datetime.datetime(int(ymd[0:4]),int(ymd[4:6]),int(ymd[6:8]))
    date = ymd_trans.strftime("%Y-%m-%d")
    beam_lst = list(f)[2:-1]
    alt13_df = pd.DataFrame()
    for beam in beam_lst:
        f_beam = f[beam]
        lat = f_beam['segment_lat'][:]
        long = f_beam['segment_lon'][:]
        ws = f_beam['ht_water_surf'][:]
        ws_sd = f_beam['stdev_water_surf'][:]
        ws_err = f_beam['err_ht_water_surf'][:]
        ortho = f_beam['ht_ortho'][:]
        wd = f_beam['water_depth'][:]
        df_beam = pd.DataFrame({'Beam': beam ,'RGT':rgt,'Cycle': cycle, 'Date':date, 'Date_num':int(ymd),'Ver.':version,
                                 'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                                'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                                'WaterD':wd})
        alt13_df = alt13_df.append(df_beam, ignore_index = True)
    
    alt13_df_out = pd.DataFrame()
    for beam_type in beam_lst:
        df_select = alt13_df.loc[alt13_df['Beam']==beam_type]
        alt13_df_out = alt13_df_out.append(out_rmv(df_select,'SurfaceH'), ignore_index = True)
        
    return alt13_df_out

In [104]:
### I made a function for stacking the information
def alt13_a_out(db,var2):
    alt13_df = pd.DataFrame()
    date_list = db['Date_num'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date_num']==prd]
        beam_lst = db['Beam'].unique()
        for beam_type in beam_lst:
            df_select = s_db.loc[s_db['Beam']==beam_type]
            alt13_df = alt13_df.append(out_rmv(df_select,var2), ignore_index = True)
    return alt13_df

In [108]:
a_tracks_out=alt13_a_out(a_tracks,'SurfaceH')

Compared the numbers

In [109]:
len(a_tracks_out)

85630

In [102]:
len(a_tracks)

88535

#### Function for making the averaged heights

In [116]:
print(statistics.mean(test['SurfaceH']))

-15.073182775619182


In [118]:
print(test['Date'].unique()[0])

2019-03-01


In [123]:
### I made a function for averaginf the height
def alt13_mean(db,var2):
    mean_df_list = pd.DataFrame()
    date_list = db['Date_num'].unique()
    for prd in date_list:
        s_db = db.loc[db['Date_num']==prd]
        av_height = statistics.mean(s_db[var2])
        mean_df = pd.DataFrame({'Av_level':av_height,'Date':s_db['Date'].unique(), 'Date_num':s_db['Date_num'].unique()})
        mean_df_list = mean_df_list.append(mean_df, ignore_index = True)
    return mean_df_list

In [126]:
av_h = alt13_mean(a_tracks_out,'SurfaceH')

In [129]:
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.plot(av_h['Date'],av_h['Av_level'],'+',markersize=4, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Plot segments and segments without outliers

In [142]:
fig=plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)
ax.grid()
ax.xaxis.set_major_locator(plt.MaxNLocator(5))
ax.set_ylim([-20,0])
ax.plot(a_tracks['Date'],a_tracks['SurfaceH'],'.', color = 'orange', markersize=0.25, label='all segements')
ax.plot(a_tracks_out['Date'],a_tracks_out['SurfaceH'],'.', color = 'green',markersize=0.4, label='all segements')
ax.plot(av_h['Date'],av_h['Av_level'],'+', color = 'red',markersize=3, label='all segements')
h_leg=ax.legend()
plt.title('Water Surface')
ax.set_xlabel('Date')
ax.set_ylabel('Water Surface, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …